In [1]:
import pandas as pd
import numpy as np 
import unicodedata
import re
from difflib import get_close_matches

In [2]:
df = pd.read_csv("train.csv", low_memory=False)
df

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65123


> Görüldüğü gibi çok fazla boş değerimiz var. Dolayısıyla, ilk önce <mark>boş değer analizi</mark> yapılacak.

In [3]:
df_nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False), columns=['Kayıp veri miktarı'])
df_nulls['% Kayıp veri miktarı'] = df.isnull().sum().sort_values(ascending=False)/len(df)
df_nulls

,Kayıp veri miktarı,% Kayıp veri miktarı
"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",64706,0.993566
Lise Bolum Diger,63599,0.976568
Lise Adi Diger,60392,0.927324
Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,50526,0.775831
Hangi STK'nin Uyesisiniz?,49518,0.760353
Uye Oldugunuz Kulubun Ismi,48691,0.747655
Baska Kurumdan Aldigi Burs Miktari,44162,0.678111
Burs Aldigi Baska Kurum,43977,0.675271
Burslu ise Burs Yuzdesi,42685,0.655432
Daha Once Baska Bir Universiteden Mezun Olmus,37345,0.573436


# Verilerin nasıl işleneceği hakkında ön planlama
## Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite: [x]
- Boş ise mezun değil olarak doldurulacak.
- Bütün üniversite isimleri küçültülecek.
- Hatalı yazılan üniversite isimleri teyit edilerek düzeltilecek.

## Lise Bolum Diger: [x]
- NLP teknikleri kullanıcak.
- 'Lise Bolumu' ile birleştirilecek.
- Boş değerler bilinmiyor.

## Lise Adi Diger: [x]
- İsimler küçültülecek.
- NLP teknikleri uygulanacak.
- 'Lise Adi' ile birleştirilecek.
    - Hala boş değerler var ise lise okumamış.

## Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?: [x]
- NLP ile anahtar kelime çıkartılacak.
- Boş ise girişimcilik deneyimi yok.
- Cümleleri Langchain ile ayrıştır. (dene).
  
## Hangi STK'nin Uyesisiniz?: [x]
- NLP ile anahtar kelime çıkartılacak.
- Boş ise 'üyelik yok'.
  
## Uye Oldugunuz Kulubun Ismi: [x]
- Boş ise 'üyelik yok'.

## Baska Kurumdan Aldigi Burs Miktari: [ ]
- Cümle içindeki sayısı al. (re.findall()).
- Boş ise burs '0 TL'.

## Burs Aldigi Baska Kurum:
- Boş ise bir kurumdan 'yok'.

## Burslu ise Burs Yuzdesi: [x]
- Boş ise burs almıyor.

## Daha Once Baska Bir Universiteden Mezun Olmus: [x]
- Boş ise 'Hayır'.

## Ingilizce Seviyeniz?: [x]
- Boş ise 'bilmiyor'.

## Anne Sektor:
- 0 - ve Boş değerlere KNN uygulanacak
- Küçük harf yapılacak.

## Spor Dalindaki Rolunuz Nedir?: [x]
- '0', '-' ve 'nan' ise '0'.
- Küçük harf yapılacak.

## Stk Projesine Katildiniz Mi?: [x]
- Boş değerler '0' olarak yazılacak.

## Baba Sektor:
- 0 - ve Boş değerlere KNN uygulanacak
- Küçük harf yapılacak.

## Aktif olarak bir STK üyesi misiniz?: [x] 
- Boş değerler 'Hayır' olarak yazılacak.

## Girisimcilikle Ilgili Deneyiminiz Var Mi?: [x]
- Boş değerler 'Hayır' olarak yazılacak.

## Baba Calisma Durumu: 
- Baba sektor'de veri var ise çalışma durumu 'Evet' yapılacak.
- KNN uygulanacak.

## Anne Calisma Durumu:
- Anne sektor'de veri var ise çalışma durumu 'Evet' yapılacak.
- KNN uygulanacak.

# Ingilizce Biliyor musunuz?: [x]
- Boş değerlere 'Hayır' yazılacak.

## Lise Mezuniyet Notu: [x]
- 4'lük yazılanları 100'lüğe çevir.
- Boş değerlere 'Not ortalaması yok' yazılacak.

## Universite Not Ortalamasi: [x]
- Küçük harf yapılacak.
- 'Not ortalaması yok' yerine 'ortalama bulunmuyor' yazılacak.
- Boş değerler ve 'Hazırlığım' yazanlara 'ortalama bulunmuyor' yazılacak.

## Ikametgah Sehri: [x]
- Küçük harf yapılacak.
- ',' veya '-' veya '/' varsa bunlar ve bunlardan önce yazılan her şey silinecek.
- Boş değerler, '------' ve 'OpenAIrequestfailedtimeout' yazanlar 'Lise Sehir'ine göre belirlenecek.

## Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?: [x]
- Boş değerlere 'Hayır' yazılacak.

## Kardes Sayisi: [x]
- Boş değerlere '0' yazılacak.
- float şeklinde yazılanlarda '.' ve sonrasındaki sayı silinecek.
- Cümle içindeki sayısı al. (re.findall()).

## Profesyonel Bir Spor Daliyla Mesgul musunuz: [x]
- Boş değerlere 'Hayır' yazılacak.

## Lise Sehir: [x]
- Küçük harf yapılacak.
- ',' veya '-' veya '/' varsa bunlar ve bunlardan önce yazılan her şey silinecek.
- Boş değerlere 'ikametgah' yaz.
  
## Lise Bolumu: [x]
- hepsi küçük harf yapılacak.
- Lise Bolum Diger ile birleştirilecek.
- NLP ile anahtar kelime çıkarılacak.

## Baba Egitim Durumu:
- Küçük harf yapılacak.
- Boş değer ve '0' 'Eğitim Yok' olarak yazılacak.
  
## Lise Turu:
- Hepsi küçük harf yapılacak.
- NLP ile 'Lise Bolumu' verisi 'Lise Turu' listesinde var ise veya [meslek lisesi bölümleri listesinde](https://www.sozcu.com.tr/meslek-lisesi-bolumleri-nelerdir-meslek-lisesinde-hangi-bolumler-var-szcu6-wp5144779) var ise  'Lise Turu' 'Meslek lisesi' ile doldurulacak.
- Boşlar KNN ile doldurulacak

## Dogum Yeri: [x]
- Boş değerlere 'ikametgah' değerleri yazılacak
- '/' veya '-' varsa bun ve bundan önce yazılan her şey silinecek.
  
## Anne Egitim Durumu:
- Hepsi küçük harf yapılacak.
- KNN ile doldurulacak

## Lise Adi: [x]
- Hepsi küçük harf yapılacak.

## Universite Kacinci Sinif: [x]
- Hepsi küçük harf yapılacak.
- 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite' var veya Daha Once Baska Bir Universiteden Mezun Olmus 'evet' ise 'Mezun'

## Universite Turu: 
- Hepsi küçük harf yapılacak.
- Boş değerlere 'okumuyor'

## Bölüm:
- Hepsi küçük harf yapılacak.
- KNN uygulanacak.

## Baska Bir Kurumdan Burs Aliyor mu?: [x]
- Boş değerlere KNN uygulanacak

## Dogum Tarihi: [x]
- Ayın ismini yazanlarla sayıları değiştir.
- datetime'a dönüştür
- aykırı ve boş verileri temizle
- eğer kayıp fazla ise KNN veya üniversite sınıfı verilerini kullanarak doldur

## Cinsiyet:
- Boş değer olanlara 'Belirtmek istemiyorum' yazılacak

## Universite Adi:
- Hepsi küçük harf yapılacak.

## Degerlendirme Puani:
- Boş değer olanı sil.

## Basvuru Yili: [x]
- datetime'a dönüştür

## Bütün verilere uygulanacak işlemler
- Hepsi küçük harf yapılacak.
- Aykırı değerler silinecek.
- numerik değerler float veya int tipine dönüştürülecek.

> Veri setinde yinelenen değer <mark>bulunmamakta</mark>.

In [4]:
df[df.duplicated()] 

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id


> Veri setimiz hem <mark>kategorik</mark> hem de <mark>numerik</mark> değerlerden oluşmaktadır.
Numerik değer ve kategorik değerlerin analizi yapılacak.

In [5]:
df.dtypes

Basvuru Yili                                                   int64
Degerlendirme Puani                                          float64
Cinsiyet                                                      object
Dogum Tarihi                                                  object
Dogum Yeri                                                    object
Ikametgah Sehri                                               object
Universite Adi                                                object
Universite Turu                                               object
Burslu ise Burs Yuzdesi                                      float64
Burs Aliyor mu?                                               object
Bölüm                                                         object
Universite Kacinci Sinif                                      object
Universite Not Ortalamasi                                     object
Daha Once Baska Bir Universiteden Mezun Olmus                 object
Lise Adi                          

In [6]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

In [7]:
df['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'] = df['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'].fillna("mezun değil")

In [8]:
df['Uye Oldugunuz Kulubun Ismi'] = df['Uye Oldugunuz Kulubun Ismi'].fillna('üyelik yok')

In [9]:
df['Burslu ise Burs Yuzdesi'].fillna(np.float64(0))

0        100.0
1        100.0
2        100.0
3        100.0
4        100.0
         ...  
65120      0.0
65121      0.0
65122      0.0
65123      0.0
65124      0.0
Name: Burslu ise Burs Yuzdesi, Length: 65125, dtype: float64

In [10]:
df['Daha Once Baska Bir Universiteden Mezun Olmus'] = df['Daha Once Baska Bir Universiteden Mezun Olmus'].fillna('hayır')

In [11]:
#df[(df['Ingilizce Seviyeniz?'] == '0') & (df['Ingilizce Biliyor musunuz?'] == 'Evet')]
#df[(df['Ingilizce Seviyeniz?'].isna()) & (df['Ingilizce Biliyor musunuz?'] == 'Evet')]
df['Ingilizce Seviyeniz?'] = df['Ingilizce Seviyeniz?'].fillna('0')

In [12]:
replace_roles = {
    "kaptan / li̇der": "kaptan",
    "lider/kaptan": "kaptan",
    "diğer": "di̇ğer",
    "-": '0',
}

df['Spor Dalindaki Rolunuz Nedir?'] = df['Spor Dalindaki Rolunuz Nedir?'].replace(replace_roles)
df['Spor Dalindaki Rolunuz Nedir?'] = df['Spor Dalindaki Rolunuz Nedir?'].fillna('0')
df['Spor Dalindaki Rolunuz Nedir?'].unique()

array(['di̇ğer', '0', 'kaptan', 'bireysel', 'takım oyuncusu'],
      dtype=object)

In [13]:
df['Stk Projesine Katildiniz Mi?'] = df['Stk Projesine Katildiniz Mi?'].fillna('hayır')

In [14]:
df['Aktif olarak bir STK üyesi misiniz?'].fillna('hayır')

0        hayır
1        hayır
2        hayır
3        hayır
4        hayır
         ...  
65120    hayır
65121    hayır
65122    hayır
65123    hayır
65124     evet
Name: Aktif olarak bir STK üyesi misiniz?, Length: 65125, dtype: object

In [15]:
df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna('hayır')

In [16]:
df['Ingilizce Biliyor musunuz?'] = df['Ingilizce Biliyor musunuz?'].fillna('hayır')

In [17]:
df['Baska Bir Kurumdan Burs Aliyor mu?'] = df['Baska Bir Kurumdan Burs Aliyor mu?'].replace(np.nan, 'hayır')
df['Baska Bir Kurumdan Burs Aliyor mu?'].unique()

array(['hayır', 'evet'], dtype=object)

In [18]:
df['Lise Mezuniyet Notu'] =  df['Lise Mezuniyet Notu'].fillna('not ortalaması yok')

lise_notes = {
    'not ortalaması yok': 0,
    '2.50 ve altı': 1,
    '0 - 24': 1,
    '0 - 25': 1,
    '44-0': 1,
    '25 - 49': 1,
    '25 - 50': 1,
    '54-45': 2,
    '69-55': 3,
    '3.00-2.50': 3,
    '50 - 74': 4,
    '50 - 75' : 4,
    '3.50-3.00': 4,
    '3.50-3': 4,
    '84-70': 5,
    '75 - 100': 5,
    '100-85': 5,
    '3.00 - 4.00': 5,
    '4.00-3.50': 5,
}

df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].replace(lise_notes)
df['Lise Mezuniyet Notu'].unique()

/tmp/ipykernel_13887/57959371.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].replace(lise_notes)


array([4, 5, 3, 1, 0, 2])

In [19]:
uni_notes = {
    'ortalama bulunmuyor': 0,
    'not ortalaması yok': 0,
    'hazırlığım': 0,
    '1.00 - 2.50': 1,
    '2.50 ve altı':  1,
    '1.80 - 2.49' : 1,
    '0 - 1.79': 1,
    '2.00 - 2.50': 1,
    '2.50 - 2.99': 2,
    '2.50 - 3.00': 2,
    '2.50 -3.00': 2,
    '3.00-2.50': 2,
    '3.50-3': 3,
    '3.00 - 3.50': 3,
    '3.00 - 3.49': 3,
    '3.00 - 4.00': 4,
    '3.50 - 4.00': 4,
    '4-3.5': 4,
    '4.0-3.5': 4,
}
df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].fillna('not ortalaması yok')
df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].replace(uni_notes)
df['Universite Not Ortalamasi'].unique()

/tmp/ipykernel_13887/1041912101.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].replace(uni_notes)


array([3, 2, 0, 1, 4])

In [20]:
df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna('hayır')

In [21]:
df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna('hayır')

In [22]:
df['Universite Kacinci Sinif'] = df['Universite Kacinci Sinif'].fillna(0)

replace_grade = {
    'tez': 'yüksek lisans'
}

df['Universite Kacinci Sinif'] = df['Universite Kacinci Sinif'].replace(replace_grade)

In [125]:
replace_lise_turu = {
    "özel": "özel lise",
    "özel lisesi": "özel lise",
    "meslek": "meslek lisesi",
}
df['Lise Turu'] = df['Lise Turu'].replace(replace_lise_turu)

# devlet  ve diger olanların 'Lise Adi'na bak
df.loc[(df['Lise Turu']=='devlet') | (df['Lise Turu']=='diger') & 
    (df['Lise Adi'].str.contains('anadolu'))] = 'anadolu lisesi'

df.loc[(df['Lise Turu']=='devlet') | (df['Lise Turu']=='diger') & 
    (df['Lise Adi'].str.contains('fen'))] = 'fen lisesi'

df.loc[(df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer') & 
    (df['Lise Adi'].str.contains('açık|acık|acik|acık|açik'))] = 'düz lise'

df.loc[(df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer') & 
    (df['Lise Adi'].str.contains('imam|ihl|i.h.l|ih'))] = 'i̇mam hatip lisesi'

df.loc[(df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer') & 
    (df['Lise Adi'].str.contains('meslek|mtal|m.t.a.l|mes|mes.|atml|a.t.m.l|mesleki'))] = 'meslek lisesi'

df['Lise Turu'].unique()

KeyError: 'Lise Adi'

In [23]:
df = df.drop(["Lise Adi", "Lise Adi Diger"], axis=1)
df.shape

(65125, 42)

In [24]:
replace_bolum = {
    'düz lise bölüm yok': 'alan yok',
    'alansız': 'alan yok',
    'alan ayrımı yok': 'alan yok',
    'alan ayrımı.olmayan lise': 'alan yok',
    'bölüm ayrımı olmayanlar': 'alan yok',
    'bölüm ayrımı bulunmamaktadır.': 'alan yok',
    'alan ayrimi olmayanlar': 'alan yok',
    '-': 'alan yok',
    'yok': 'alan yok',
    'alan bulunmayan özel lise': 'alan yok',
    '': 'alan yok',
    'alansız': 'alan yok',
    'bölüm yok': 'alan yok',
    'alan ayrimi olmayanlar': 'alan yok',
    '_': 'alan yok',
    '.': 'alan yok',
    'bölüm yoktu': 'alan yok',
    'bolum yok': 'alan yok',
    'bölüm ayrımı olmayan': 'alan yok'
}

df['Lise Bolumu'] = df['Lise Bolumu'].replace(replace_bolum)

df.loc[df['Lise Bolumu'].isnull() & df['Lise Bolum Diger'].notnull(), 'Lise Bolumu'] = df['Lise Bolum Diger']

df = df.dropna(subset=['Lise Bolumu'])

filtered_sayisal = df['Lise Bolumu'].str.contains("fen|mf|sayısal|sayisal|fm|m-f")
filtered_ea = df['Lise Bolumu'].str.contains("eşit ağırlık|esit agırlık|ea|tm|türkçe-matematik|türkçe matematik|turkce matematik|eşit agırlık|esıt agırlık|türkçe-mat|esi̇t agirlik|eşit agırlık|eşit agırlık|türkçe-mat|turkce-matematik|eşi̇t agirlik|türkçe -matematik|e-a|türkçe ve matematik|eşi̇t ağirlik|eşitağırlık|türkçe- matematik|turkce-matematık|turkce ve matematık|matematik-türkçe|turkce matemati̇k|esitagirlik|türkçe - matematik|t-m|t.m")
filtered_ts = df['Lise Bolumu'].str.contains("sözel|sozel|türkçe-sosyal|türkçe sosyal|sosyal bilimler|ts|t/s")
filtered_dil = df['Lise Bolumu'].str.contains("dil|ingilizce|yabancı di̇l")

combine = pd.concat([filtered_sayisal, filtered_ea, filtered_ts, filtered_dil])

filtered_another = df[~df['Lise Bolumu'].isin(combine)]

df.loc[df['Lise Turu'] == 'meslek lisesi', 'Lise Bolumu'] = 'diğer'
df.loc[filtered_sayisal, 'Lise Bolumu'] = 'sayısal'
df.loc[filtered_ea, 'Lise Bolumu'] = 'eşit ağırlık'
df.loc[filtered_ts, 'Lise Bolumu'] = 'sözel'
df.loc[filtered_dil, 'Lise Bolumu'] = 'dil'

invalid_ids = df.loc[~df['Lise Bolumu'].isin(['diğer', 'sayısal', 'eşit ağırlık', 'sözel', 'dil']), 'id'].tolist()
df = df[~df['id'].isin(invalid_ids)]

df['Lise Bolumu'].unique()


array(['sayısal', 'diğer', 'eşit ağırlık', 'sözel', 'dil'], dtype=object)

In [68]:
df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].fillna(0)

filtered = df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].str.contains('girişimim yok|yer almak cok isterim|görev almayı çok isterim|deneyimim yok', na=False)

df.loc[filtered, 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = 0

df.loc[~filtered, 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = 1

df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].unique()


array([1, 0], dtype=object)

In [72]:
df = df.drop(['Hangi STK\'nin Uyesisiniz?'], axis=1)
df.shape

(63495, 41)

In [27]:
tr = pd.read_csv("il_ilce.csv")

In [28]:
for col in tr.columns:
    if tr[col].dtype == 'object':
        tr[col] = tr[col].str.lower()
tr["il"].unique()

array(['adana', 'adıyaman', 'afyonkarahisar', 'aksaray', 'amasya',
       'ankara', 'antalya', 'ardahan', 'artvin', 'aydın', 'ağrı',
       'balıkesir', 'bartın', 'batman', 'bayburt', 'bilecik', 'bingöl',
       'bitlis', 'bolu', 'burdur', 'bursa', 'denizli', 'diyarbakır',
       'düzce', 'edirne', 'elazığ', 'erzincan', 'erzurum', 'eskişehir',
       'gaziantep', 'giresun', 'gümüşhane', 'hakkâri', 'hatay', 'isparta',
       'iğdır', 'kahramanmaraş', 'karabük', 'karaman', 'kars',
       'kastamonu', 'kayseri', 'kilis', 'kocaeli', 'konya', 'kütahya',
       'kırklareli', 'kırıkkale', 'kırşehir', 'malatya', 'manisa',
       'mardin', 'mersin', 'muğla', 'muş', 'nevşehir', 'niğde', 'ordu',
       'osmaniye', 'rize', 'sakarya', 'samsun', 'siirt', 'sinop', 'sivas',
       'tekirdağ', 'tokat', 'trabzon', 'tunceli', 'uşak', 'van', 'yalova',
       'yozgat', 'zonguldak', 'çanakkale', 'çankırı', 'çorum',
       'i̇stanbul', 'i̇zmir', 'şanlıurfa', 'şırnak', 'kosova', 'bakü',
       'kıbrıs', 'alma

In [29]:
"""tr2 = tr.set_index('ilce')

def process_cities(df, data, fill_data, data_map):
    df[data] = df[data].str.replace('ı', 'i', regex=False)
    df.loc[df[data].isnull() & df[fill_data].notnull(), data] = df[fill_data]
    
    
    df = df.dropna(subset=[data])

    df[data] = df[data].apply(
        lambda x: unicodedata.normalize('NFKD', x.strip().lower()).encode('ASCII', 'ignore').decode('ASCII')
    )

    df[data] = df[data].apply(
        lambda x: re.split(r'[\/\-,~ ]', x)[-1].strip()
    )

    df[data] =  df[data].replace(data_map)

    ids_to_drop = []

    for city, id_ in zip(df[data], df["id"]):
        if city not in tr["il"]: # verisetinde olmamasının nedini yazım yanlışlığı olabilir
            match_city = get_close_matches(city, tr2["il"]) # yazımı kontrol et
            if match_city: # match bulundu ise
                df.loc[df["id"]==id_, data] = match_city[0] # yazımı düzelt
            else: # şehir match'ı bulunmadı 
                match_district = get_close_matches(city, tr["ilce"]) # tr'deki ilçeleri kontrol et
                if match_district: # ilçe varsa
                    province = tr.loc[tr['ilce'] == match_district[0], 'il'].values  
                    if len(province) > 0: # ilçe verisetinde var ise
                        df.loc[df["id"] == id_, data] = province[0]
                    else: # ilçe, veri setinde yok ise
                        print("TR veri setinde yok", id_, "   ", city)
                else: # ilçe match'ı bulunmadıysa
                    ids_to_drop.append(id_)
                    print("MATCH YOK: ", id_, "   ", city)
            
    df = df[~df["id"].isin(ids_to_drop)]
    return df"""


<>:1: SyntaxWarning: invalid escape sequence '\/'
<>:1: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipykernel_13887/1851444829.py:1: SyntaxWarning: invalid escape sequence '\/'
  """tr2 = tr.set_index('ilce')


'tr2 = tr.set_index(\'ilce\')\n\ndef process_cities(df, data, fill_data, data_map):\n    df[data] = df[data].str.replace(\'ı\', \'i\', regex=False)\n    df.loc[df[data].isnull() & df[fill_data].notnull(), data] = df[fill_data]\n    \n    \n    df = df.dropna(subset=[data])\n\n    df[data] = df[data].apply(\n        lambda x: unicodedata.normalize(\'NFKD\', x.strip().lower()).encode(\'ASCII\', \'ignore\').decode(\'ASCII\')\n    )\n\n    df[data] = df[data].apply(\n        lambda x: re.split(r\'[\\/\\-,~ ]\', x)[-1].strip()\n    )\n\n    df[data] =  df[data].replace(data_map)\n\n    ids_to_drop = []\n\n    for city, id_ in zip(df[data], df["id"]):\n        if city not in tr["il"]: # verisetinde olmamasının nedini yazım yanlışlığı olabilir\n            match_city = get_close_matches(city, tr2["il"]) # yazımı kontrol et\n            if match_city: # match bulundu ise\n                df.loc[df["id"]==id_, data] = match_city[0] # yazımı düzelt\n            else: # şehir match\'ı bulunmadı \

In [30]:
"""replace_ikametgah = {
    "ş.urfa": "şanlıurfa",
    "k.maraş": "kahramanmaraş"
}

df = process_cities(df, 'Ikametgah Sehri', 'Lise Sehir', replace_ikametgah)"""

'replace_ikametgah = {\n    "ş.urfa": "şanlıurfa",\n    "k.maraş": "kahramanmaraş"\n}\n\ndf = process_cities(df, \'Ikametgah Sehri\', \'Lise Sehir\', replace_ikametgah)'

In [31]:
"""replace_lise_sehir = {
    "ist." : "istanbul",
    "istanbulkucukcekmece": "istanbul",
    "ş.urfa": "şanlıurfa",
    "k.maraş": "kahramanmaraş"
}

df = process_cities(df, 'Lise Sehir', 'Ikametgah Sehri', replace_lise_sehir)
df = df[~df['Lise Sehir'].isin(['', 'shkoder', 'asdasd'])]"""

'replace_lise_sehir = {\n    "ist." : "istanbul",\n    "istanbulkucukcekmece": "istanbul",\n    "ş.urfa": "şanlıurfa",\n    "k.maraş": "kahramanmaraş"\n}\n\ndf = process_cities(df, \'Lise Sehir\', \'Ikametgah Sehri\', replace_lise_sehir)\ndf = df[~df[\'Lise Sehir\'].isin([\'\', \'shkoder\', \'asdasd\'])]'

In [32]:
"""replace_dy = {
    "ş.urfa": "şanlıurfa",
    "k.maras": "kahramanmaraş",
    "k.maraş": "kahramanmaraş",
    "ist." : "istanbul",
    "eyup": "eyüpsultan",
    "gumushacikoy": "gümüşhacıköy",
    "cicekdagi": "çiçekdağı",
    "sevastopol": "sivastopol",
    "yigilca": "yığılca"
}

df = process_cities(df, 'Dogum Yeri', 'Ikametgah Sehri', replace_dy)"""

'replace_dy = {\n    "ş.urfa": "şanlıurfa",\n    "k.maras": "kahramanmaraş",\n    "k.maraş": "kahramanmaraş",\n    "ist." : "istanbul",\n    "eyup": "eyüpsultan",\n    "gumushacikoy": "gümüşhacıköy",\n    "cicekdagi": "çiçekdağı",\n    "sevastopol": "sivastopol",\n    "yigilca": "yığılca"\n}\n\ndf = process_cities(df, \'Dogum Yeri\', \'Ikametgah Sehri\', replace_dy)'

In [33]:
replace_sentence = {
    'kardeş sayısı 1 ek bilgi aile hk. anne vefat': 1
}

df["Kardes Sayisi"] = df["Kardes Sayisi"].replace(replace_sentence)

df['Kardes Sayisi'] = pd.to_numeric(df['Kardes Sayisi'])

df["Kardes Sayisi"] = df["Kardes Sayisi"].fillna(0)

df["Kardes Sayisi"] = df["Kardes Sayisi"].astype(int)

df["Kardes Sayisi"].unique()

array([ 1,  6, 10,  3,  0,  4,  2,  5,  7,  8, 11,  9, 12, 13, 14])

In [34]:
df['Basvuru Yili'].unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [51]:
df['Basvuru Yili'] = pd.to_datetime(df['Basvuru Yili'], format='%Y')

In [52]:
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='\\.', value="/", regex=True)

In [53]:
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='ocak', value='/01/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='subat', value='/02/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='mart', value='/03/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='nisan', value='/04/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='mayis', value='/05/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='haziran', value='/06/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='temmuz', value='/07/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='ağustos', value='/08/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='eylül', value='/09/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='ekim', value='/10/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='kasim', value='/11/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='aralik', value='/12/', regex=True)

In [54]:
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='\\_', value="", regex=True)

In [55]:
df['Dogum Tarihi'] = pd.to_datetime(df['Dogum Tarihi'], format='mixed', errors='coerce')

In [56]:
df.loc[df['Dogum Tarihi'].isnull() == True].shape

(0, 42)

In [57]:
df.dropna(subset='Dogum Tarihi', inplace=True)

In [110]:
df_deneme = df.copy()
df_deneme['Baska Kurumdan Aldigi Burs Miktari']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
            ...     
65120              -
65121              -
65122    500₺ - 999₺
65123              -
65124    500₺ - 999₺
Name: Baska Kurumdan Aldigi Burs Miktari, Length: 63495, dtype: object

In [111]:
df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'] = df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'].str.findall('\d+').fillna("-")

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_13887/297223768.py:11: SyntaxWarning: invalid escape sequence '\d'
  ), 'Baska Kurumdan Aldigi Burs Miktari'].str.findall('\d+').fillna("-")


In [112]:
df_deneme.drop(df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df_deneme['Baska Kurumdan Aldigi Burs Miktari'].str.len() == 0)
                                , 'Baska Kurumdan Aldigi Burs Miktari'].index, inplace=True)

In [113]:
#df_deneme.loc[df_deneme.id == 57067, 'Baska Kurumdan Aldigi Burs Miktari']

In [114]:
df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'] = df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'].apply(lambda x: '-' if x == '0' else max(x))

In [115]:
df_deneme.drop(df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df_deneme['Baska Kurumdan Aldigi Burs Miktari'].str.len() == 0)
                                , 'Baska Kurumdan Aldigi Burs Miktari'].index, inplace=True)

In [116]:
#TÜFE değrlerine göre 2023 baz alınarak tahmini enflasyon
tufe=pd.DataFrame(
    {
        'yil': [2014,2015,2016,2017,2018,2019,2020,2021,2022],
        'tufe': [5.17,4.81,4.49,4.02,3.46,3.02,2.70,2.34,1.58]
    }
)
tufe['yil'] = pd.to_datetime(tufe['yil'], format='%Y')
tufe

,yil,tufe
0,2014-01-01,5.17
1,2015-01-01,4.81
2,2016-01-01,4.49
3,2017-01-01,4.02
4,2018-01-01,3.46
5,2019-01-01,3.02
6,2020-01-01,2.70
7,2021-01-01,2.34
8,2022-01-01,1.58


In [117]:
for a in tufe.index:
    df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df_deneme['Basvuru Yili'].dt.year == tufe.iloc[a].yil.year)
                                 , 'Baska Kurumdan Aldigi Burs Miktari'] = df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df_deneme['Basvuru Yili'].dt.year == tufe.iloc[a].yil.year)
                                 , 'Baska Kurumdan Aldigi Burs Miktari'].apply(lambda x: tufe.iloc[a].tufe* int(x)).apply(lambda x: '-' if int(x) <= 10 else (
                                                                               '0 - 499₺' if (int(x) > 10) & (int(x) <= 500) else (
                                                                                   '500₺ - 999₺' if (int(x) > 500) & (int(x) < 1000) else '1000₺ ve üstü')
                                                                               ))

In [118]:
df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                                 , 'Baska Kurumdan Aldigi Burs Miktari'] = df_deneme.loc[
                        ~((df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df_deneme['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df_deneme['Baska Kurumdan Aldigi Burs Miktari'])
                                 , 'Baska Kurumdan Aldigi Burs Miktari'].apply(lambda x: '-' if int(x) <= 10 else (
                                                                               '0 - 499₺' if (int(x) > 10) & (int(x) <= 500) else (
                                                                                   '500₺ - 999₺' if (int(x) > 500) & (int(x) < 1000) else (
                                                                                       '1000₺ ve üstü' if int(x) >= 1000 else 'hata')
                                                                               ))
                                                                              )

In [119]:
df_deneme['Baska Kurumdan Aldigi Burs Miktari'].unique()

array(['-', '1000₺ ve üstü', '500₺ - 999₺', '0 - 499₺'], dtype=object)

In [93]:
!git add .
!git commit -m "berke"
!git push

[main 3183119] berke
 3 files changed, 704 insertions(+), 686 deletions(-)
Nesneler ortaya dökülüyor: 9, bitti.
Nesneler sayılıyor: 100% (9/9), bitti.
Delta sıkıştırması 8 iş parçacığı kullanıyor
Nesneler sıkıştırılıyor: 100% (5/5), bitti.
Nesneler yazılıyor: 100% (5/5), 6.51 KiB | 6.51 MiB/sn, bitti.
Toplam 5 (delta 4), yeniden kullanılan 0 (delta 0), yeniden kullanılan paket 0 (0 konumundan)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/DALLI-KAKTUS/btk-datathon-2024.git
   af43fa1..3183119  main -> main
